In [1]:
using OceananigansLagrangianFilter: create_filtered_vars, create_forcing, create_output_fields, initialise_filtered_vars
using OceananigansLagrangianFilter
using Oceananigans.Units
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CUDA: CuArray
using CairoMakie 
using NCDatasets
using Printf

Precompiling packages...
   3563.2 ms  ✓ OceananigansLagrangianFilter
  1 dependency successfully precompiled in 5 seconds. 356 already precompiled.


In [2]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L_front/H
Ro = 0.1             # Rossby number (defines M^2)

# Derived parameters
H_front = L_front/aspect_ratio
M² = (Ro^2*f^2*L_front)/H_front
Δb = M²*L_front
κh = 1e-4
κv = 1e-7

filename = "periodic_geostrophic_adjustment"

"periodic_geostrophic_adjustment"

In [3]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), 
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [4]:
# Define closure
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

# Define tracers
tracers = (:b,:T)

# Define forcing
forcing = NamedTuple()



NamedTuple()

In [5]:
# Define filter configuration
filter_params = (;a1 = 1, c1 = 1, N_coeffs = 0.5)
filter_config = OnlineFilterConfig( grid = grid,
                                    var_names_to_filter = ("b","T"), 
                                    velocity_names = ("u","w"),
                                    filter_params = filter_params,
                                    compute_Eulerian_filter = true)

OnlineFilterConfig(300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25, ("b", "T"), ("u", "w"), (a1 = 1, c1 = 1, N_coeffs = 0.5), true, 5, true)

In [6]:
# Now we define the filtering tracers and forcings
# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(filter_config)
@info "Created filtered variables: $filtered_vars"
tracers = (filtered_vars..., tracers...)

# Create forcing for these filtered variables
filter_forcing = create_forcing(filtered_vars, filter_config)
@info "Created forcing for filtered variables"

forcing = merge(forcing, filter_forcing)


[ Info: Created filtered variables: (:b_C1, :T_C1, :xi_u_C1, :xi_w_C1)
[ Info: Created forcing for filtered variables


(T_C1 = (ContinuousForcing{Tuple{Int64}}
├── func: #13 (generic function with 1 method)
├── parameters: (1,)
└── field dependencies: (:T_C1,), ContinuousForcing{Nothing}
├── func: (::OceananigansLagrangianFilter.Utils.var"#original_var_forcing_func#23") (generic function with 1 method)
├── parameters: nothing
└── field dependencies: (var_key = :T,)), xi_u_C1 = (ContinuousForcing{Tuple{Int64, Int64}}
├── func: #19 (generic function with 1 method)
├── parameters: (1, 0)
└── field dependencies: (:u,), ContinuousForcing{Tuple{Int64}}
├── func: #13 (generic function with 1 method)
├── parameters: (1,)
└── field dependencies: (:xi_u_C1,)), xi_w_C1 = (ContinuousForcing{Tuple{Int64, Int64}}
├── func: #19 (generic function with 1 method)
├── parameters: (1, 0)
└── field dependencies: (:w,), ContinuousForcing{Tuple{Int64}}
├── func: #13 (generic function with 1 method)
├── parameters: (1,)
└── field dependencies: (:xi_w_C1,)), b_C1 = (ContinuousForcing{Tuple{Int64}}
├── func: #13 (generic functi

In [7]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = tracers,
                forcing = forcing,
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b_C1, T_C1, xi_u_C1, xi_w_C1, b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b_C1=0.0001, T_C1=0.0001, xi_u_C1=0.0001, xi_w_C1=0.0001, b=0.0001, T=0.0001))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=(b_C1=1.0e-7, T_C1=1.0e-7, xi_u_C1=1.0e-7, xi_w_C1=1.0e-7, b=1.0e-7, T=1.0e-7))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [8]:
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/20)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [9]:
simulation = Simulation(model, Δt=20minutes, stop_time=5days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [10]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [11]:

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(50))

In [13]:
# Output setup
# Create filtered outputs
outputs = create_output_fields(model, filter_config)

# Add in original variables if desired
outputs["b"] = model.tracers.b
outputs["T"] = model.tracers.T
outputs["u"] = model.velocities.u
outputs["v"] = model.velocities.v
outputs["w"] = model.velocities.w


#For Lagrangian filtering
simulation.output_writers[:jld2fields] = JLD2Writer(
    model, outputs, filename=filename * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


#for python visualisation
rm(filename * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, outputs, filename=filename * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)
    

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_geostrophic_adjustment.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 9 outputs: (v, xi_u, w, T, b, T_Lagrangian_filtered, xi_w, u, b_Lagrangian_filtered)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 28.9 KiB

In [14]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 5.218 minutes, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (23.412 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (4.023 seconds).


[09.40%] i: 50, t: 11.275 hours, wall time: 7.796 seconds, max(u): (2.399e-02, 5.051e-02, 4.262e-04) m/s, next Δt: 8.263 minutes
[12.55%] i: 100, t: 15.062 hours, wall time: 279.416 ms, max(u): (2.436e-02, 1.238e-02, 3.869e-04) m/s, next Δt: 4.097 minutes
[15.41%] i: 150, t: 18.496 hours, wall time: 262.273 ms, max(u): (1.113e-02, 2.008e-03, 2.284e-04) m/s, next Δt: 4.958 minutes
[18.49%] i: 200, t: 22.190 hours, wall time: 281.508 ms, max(u): (3.053e-02, 3.420e-02, 5.436e-04) m/s, next Δt: 3.638 minutes
[21.14%] i: 250, t: 1.057 days, wall time: 262.819 ms, max(u): (1.127e-02, 5.991e-02, 2.388e-04) m/s, next Δt: 4.401 minutes
[24.46%] i: 300, t: 1.223 days, wall time: 274.359 ms, max(u): (2.734e-02, 4.596e-02, 5.806e-04) m/s, next Δt: 4.063 minutes
[27.05%] i: 350, t: 1.352 days, wall time: 259.346 ms, max(u): (2.540e-02, 1.417e-02, 4.619e-04) m/s, next Δt: 3.917 minutes
[29.96%] i: 400, t: 1.498 days, wall time: 270.017 ms, max(u): (1.175e-02, 3.424e-03, 3.193e-04) m/s, next Δt: 5.21

[ Info: Simulation is stopping after running for 38.838 seconds.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.
[ Info: Simulation completed in 38.876 seconds
